## 6 ドメインの完全性と整合性

すべてのデータ値が常に "_正しい_" ことを保証できれば、すべての場所で値をチェックするような、 **防御的コーディング** をする必要がなくなる。

"_正しい_" とは以下のような状態である。

### 完全性 (妥当性)

それぞれのデータが正しいビジネスルールに従っているか

例:

* UnitQuantity は 1 - 1000 の間 (int ではない)。
* 注文は常に明細行が1つ必要
* 注文は発送部門に送られる前に発送先住所が検証されているべき

### 整合性

ドメインモデルの異なる部分（値同士、エンティティ同士）が事実に基づいて一致していること

例:

* 注文の合計金額は個々の行の合計と同じ
* 確定した注文には必ず請求書がある
* 注文に割引クーポンが適用されているならば、そのクーポンコードは必ず使用済みとしてマークされている

正しい値は、 **自己文書化されたコード** の作成を助け、その値に依存する処理について、 **値の要件に対するユニットテストを書く必要が一切なくなる**

## 6.1 単純型の完全性

完全性の制約を満たさない限りこれらの値を作成できないようにする。
デフォルトのコンストラクタの代わりに完全性をチェックする初期化関数を用意するアプローチを **スマートコントラクタ** と呼ぶ。

In [1]:
// 完全性が型レベルで担保されていない例 (コメントによって完全性を補足している)
// type WidgetCode = WidgetCode of string // 先頭が "W" + 数字4桁
// type UnitQuantity = UnitQuantity of int // 1 以上 1000 以下
// type KilogramQuantity = KilogramQuantity of decimal // 0.05 以上、 100.00 以下。精度は 1/100

// 完全性が型レベルで担保された型
type UnitQuantity = private UnitQuantity of int // デフォルトのコンストラクタをプライベートにして別の関数を用意する

// 型と同じ名前のサブモジュールを作成する
module UnitQuantity =
    let create qty =
        if qty < 1 then
            Error "UnitQuantity can not be negative"
        else if qty > 1000 then
            Error "UnitQuantity can not be more than 1000"
        else
            Ok (UnitQuantity qty) // サブモジュール内からはコンストラクタにアクセスできる

    // プライベートコンストラクタはパターンマッチしてデータを抽出できないので、アクセサを用意する
    let value (UnitQuantity qty) = qty

// Usage
// compile error!
// let unitQty = UnitQuantyty 1

let unitQtyResult = UnitQuantity.create 1

match unitQtyResult with
| Error msg ->
    printfn "Failure, Message is %s" msg
| Ok uQty ->
    printfn "Success. Value is %A" uQty
    let innerValue = UnitQuantity.value uQty
    printfn "innerValue is %i" innerValue

The below script needs to be able to find the current output cell; this is an easy method to get it.

Success. Value is UnitQuantity 1
innerValue is 1


## 6.2 測定単位

数値に単位を付与する。測定単位は静的に解決されるのでランタイムオーバーヘッドは発生しない。

In [2]:
[<Measure>]
type kg

[<Measure>]
type m

// usage
let fiveKilos = 5.0<kg>
let fiveMeters = 5.0<m>

// compile error!
// fiveKilos = fiveMeters

// compile error!
// let listOfWeights = [
//   fiveKilos
//   fiveMeters <- here
// ]

// 単位に加え、型で上限と下限を規定する。
type KilogramQuantity = KilogramQuantity of decimal<kg>

## 6.4 ビジネスルールを型システムで表現する

例:

* 検証済みメールと未検証メールがある場合、 **フラグの使用を避ける**。 その代わりにそれぞれを別々にモデル化する（ = 別々の型を定義する）。
* 連絡先としてメールか住所いずれかを必ずもつことを表すならば、「メールのみ」「住所のみ」「両方」の3ケースの型をつくる

In [25]:
type CustomerEmail =
    | Unverified of EmailAddress
    | Verified of VerifiedEmailAddress
and EmailAddress = EmailAddress of string
and VerifiedEmailAddress = private VerifiedEmailAddress of string // private constructor

module VerifiedEmailAddress =
    let create (EmailAddress address) =
    
        // Verification process here...
        
        Ok (VerifiedEmailAddress address)

type EmailContactInfo = Undefined
type PostalContactInfo = Undefined
type Name = Name of string

type BothContactMethods = {
    Email : EmailContactInfo
    Address : PostalContactInfo
}

type ContactInfo =
    | EmailOnly of EmailContactInfo
    | AddrOnly of PostalContactInfo
    | EmailAndAddr of BothContactMethods

type Contact = {
    Name : Name
    ContactInfo : ContactInfo
}

### 6.5.3 同じコンテキストの集約間の整合性

**関数スタイルにおいてはユースケースごとに集約を定義しても良い** 。そもそも集約の発想はオブジェクト指向の静的モデリングから発生している一方で、関数スタイルではシステムを動的な側面から捉えるため。

ただしどちらの場合でも集約がトランザクション（原子性）の境界であることは共通している。

### 6.5.4 同一データに作用する複数の集約

例: Account 集約と MoneyTransfer 集約がどちらもアカウント残高に作用し、どちらも残高がマイナスにならないようにする。

-> 型または検証関数をそれぞれの集約で共有する

> 多くの場合、型を使って制約をモデル化すれば、複数の集約間で制約を共有できます。たとえば、アカウントの残高がゼロを下回らないという要件は、NonNegativeMoney ＜非負の金額＞型でモデル化できるかもしれません。それを採用できない場合は、検証関数の共有もできます。検証関数は特定のオブジェクトに関連づけられておらず、グローバルな状態にも依存していないため、異なるワークフローで再利用するのも簡単です。